<a href="https://colab.research.google.com/github/HDietrich416/Data-Bootcamp/blob/master/Unit%2021%20-%20Machine%20Learning/Random_Forest_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Dependencies

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
import joblib

# Read the CSV and Perform Basic Data Cleaning

In [17]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()

# Convert dtypes of int64 to float64
for column, content in df.items():
    if df[column].dtype == 'int64':
        df = df.astype({column: 'float64'})

df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0.0,0.0,0.0,0.0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,9.11,2.87,-1.62,25.8,2.0,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0.0,1.0,0.0,0.0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,39.30,31.04,-10.49,76.3,1.0,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0.0,1.0,0.0,0.0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,891.96,668.95,-230.35,505.6,1.0,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0.0,0.0,0.0,0.0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,926.16,874.33,-314.24,40.9,1.0,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0.0,0.0,0.0,0.0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,0.762,0.139,-0.532,3.14020,0.06730,-0.06730,686.0,18.7,-18.7,2.77,0.90,-0.30,1160.0,427.65,420.33,-136.70,40.2,2.0,6046.0,189.0,-232.0,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [21]:
# Set features. This will also be used as your x values.
X = df.drop(["koi_disposition", "koi_steff_err2", "koi_srad_err1", "koi_slogg_err2", "koi_slogg", "koi_model_snr"], axis=1)
feature_names = df.columns
y = df["koi_disposition"]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [23]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_slogg_err1,koi_srad,koi_srad_err2,ra,dec,koi_kepmag
6122,0.0,0.0,0.0,0.0,6.768901,7.380000e-05,-7.380000e-05,133.077240,0.008440,-0.008440,0.150,0.305,-0.150,3.61600,0.3060,-0.3060,123.1,13.2,-13.2,1.24,0.34,-0.23,1017.0,253.30,204.89,-103.87,1.0,5737.0,154.0,0.153,1.125,-0.207,294.40472,39.351681,14.725
6370,0.0,1.0,0.0,1.0,0.733726,6.060000e-06,-6.060000e-06,132.020050,0.007950,-0.007950,0.291,0.193,-0.291,2.30900,0.2820,-0.2820,114.6,9.3,-9.3,0.86,0.23,-0.06,1867.0,2891.64,2253.61,-677.78,1.0,5855.0,158.0,0.033,0.797,-0.056,284.50391,42.463860,15.770
2879,1.0,0.0,0.0,0.0,7.652707,6.540000e-05,-6.540000e-05,134.460380,0.006190,-0.006190,0.970,0.879,-0.942,79.89690,0.0000,0.0000,641.1,42.9,-42.9,3.21,0.97,-0.32,989.0,226.81,195.16,-64.34,1.0,6328.0,151.0,0.050,0.963,-0.097,295.50211,38.983540,13.099
107,0.0,0.0,0.0,0.0,7.953547,1.910000e-05,-1.910000e-05,174.662240,0.001820,-0.001820,0.300,0.145,-0.300,2.63120,0.0595,-0.0595,875.4,25.3,-25.3,2.25,0.07,-0.14,696.0,55.37,7.15,-10.12,1.0,4768.0,76.0,0.056,0.779,-0.049,291.15878,40.750271,15.660
29,0.0,0.0,0.0,0.0,4.959319,5.150000e-07,-5.150000e-07,172.258529,0.000083,-0.000083,0.831,0.016,-0.004,2.22739,0.0075,-0.0075,9802.0,20.0,-20.0,12.21,1.96,-1.46,1103.0,349.40,146.52,-93.21,1.0,5712.0,77.0,0.110,1.082,-0.130,292.16705,48.727589,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [24]:
# Scale your data

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print(X.shape, y.shape)

(6991, 35) (6991,)


# Train the Model



In [25]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8964530892448512

# Select Features

In [26]:
features = sorted(zip(rf.feature_importances_, feature_names), reverse=True)
ranked_features = pd.DataFrame(features, columns=['Score', 'Column'])
ranked_features = ranked_features.set_index('Column')
ranked_features

,Score
Column,
koi_fpflag_ss,0.103173
koi_disposition,0.102009
koi_fpflag_nt,0.065179
koi_depth_err2,0.051613
koi_duration_err1,0.042434
koi_duration,0.041004
koi_tce_plnt_num,0.040231
koi_prad_err1,0.038123
koi_fpflag_co,0.037327


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [27]:
# Create the RandomSearchCV model
model_3 = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [250, 300, 350],
    'max_depth': [125, 150, 175]
}
grid = GridSearchCV(model_3, param_grid, verbose=3)

# Train the model with GridSearch
_ = grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... max_depth=125, n_estimators=250, score=0.904, total=   3.7s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.7s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.908, total=   3.6s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.3s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.891, total=   3.7s
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.877, total=   3.6s
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.885, total=   3.6s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.907, total=   4.5s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.906, total=   4.4s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.892, total=   4.4s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.873, total=   4.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  3.3min finished


In [28]:
# Train the model with GridSearch

# Tuned parameters

n_estimators = grid.best_params_['n_estimators']
max_depth = grid.best_params_['max_depth']
criterion = 'entropy'

# Tuned model
tuned_model = RandomForestClassifier(n_estimators=n_estimators, 
                                     criterion=criterion, max_depth=max_depth, random_state=42)
tuned_model.fit(X_train_scaled, y_train)

model_3_training_score = round(tuned_model.score(X_train_scaled, y_train)*100,3)
tuned_accuracy = round(tuned_model.score(X_test_scaled, y_test)*100,3)

print(f"Training Data Score: {model_3_training_score} %")
print(f"Testing Data Score: {tuned_accuracy} %")

Training Data Score: 100.0 %
Testing Data Score: 89.76 %


# Save the Model

In [29]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash

filename = 'RandomForestClassifier.sav'
joblib.dump(model_3, filename)

['RandomForestClassifier.sav']